In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch03SparkToolSet-Streaming3").getOrCreate()

In [3]:
sc = spark.sparkContext
sc._jsc.sc().uiWebUrl().get()

'http://gw02.itversity.com:4041'

In [4]:
for x in sc._conf.getAll():
    if 'PROXY' in x[0]:
        print(x[1])
print(spark.conf.get('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES'))
print(spark.conf.get('spark.driver.appUIAddress'))

rm01.itversity.com
http://rm01.itversity.com:19288/proxy/application_1533622723243_11246
http://rm01.itversity.com:19288/proxy/application_1533622723243_11246
http://gw02.itversity.com:4041


In [5]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv")

In [6]:
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [7]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [8]:
staticDataFrame.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:00|     0.85|   14075.0|United Kingdom|
|   580538|    23126|FELTCRA

In [9]:
sc.version

'2.3.0.2.6.5.0-292'

In [11]:
# COMMAND ----------
from pyspark.sql.functions import window, column, desc, col
t1 = staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

In [12]:
 t1.show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   15311.0|[2011-04-03 20:00...|1688.6500000000008|
|   15624.0|[2011-03-27 20:00...|183.89999999999998|
|   15299.0|[2011-04-13 20:00...|             -6.01|
|   12678.0|[2011-07-12 20:00...|            915.46|
|   13184.0|[2011-02-21 19:00...|212.51999999999998|
+----------+--------------------+------------------+
only showing top 5 rows



In [13]:
t1.take(1)

[Row(CustomerId=16057.0, window=Row(start=datetime.datetime(2011, 12, 4, 19, 0), end=datetime.datetime(2011, 12, 5, 19, 0)), sum(total_cost)=-37.6)]

In [14]:
t1.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[CustomerId#16, window#109], functions=[sum(total_cost#99)])
+- Exchange hashpartitioning(CustomerId#16, window#109, 200)
   +- *(1) HashAggregate(keys=[CustomerId#16, window#109], functions=[partial_sum(total_cost#99)])
      +- *(1) Project [named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) as double) = (cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) THEN (CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) + 1) ELSE CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongType) - 0) as double) / 8.64E10)) END + 0) - 1) * 86400000000) + 0), LongType, TimestampType), end, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(InvoiceDate#14, TimestampType, LongTy

In [15]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [16]:
# COMMAND ----------
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv")

In [18]:
streamingDataFrame.explain() # note the Error

== Physical Plan ==
org.apache.spark.sql.AnalysisException: Queries with streaming sources must be executed with writeStream.start();;
FileSource[/user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv]


In [19]:
# COMMAND ----------
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

In [20]:
# COMMAND ----------
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [22]:
# COMMAND ----------

spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(20)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|      null|[2010-12-02 19:00...| 23021.99999999999|
|      null|[2010-11-30 19:00...|12584.299999999988|
|   15061.0|[2010-12-01 19:00...| 9407.339999999998|
|   13777.0|[2010-11-30 19:00...|           6585.16|
|   17850.0|[2010-12-01 19:00...|3891.8699999999985|
|   16029.0|[2010-11-30 19:00...|           3702.12|
|   16210.0|[2010-11-30 19:00...|2474.7399999999993|
|   13081.0|[2010-12-02 19:00...|           2366.78|
|   16754.0|[2010-12-01 19:00...|            2002.4|
|   12433.0|[2010-11-30 19:00...|1919.1400000000008|
|   15299.0|[2010-12-01 19:00...|1835.0100000000002|
|   17511.0|[2010-11-30 19:00...|           1825.74|
|   14031.0|[2010-12-01 19:00...|1714.2500000000005|
|   14911.0|[2010-12-02 19:00...|1705.6499999999994|
|   14680.0|[2010-12-02 19:00...|1680.8799999999999|
|   13013.0|[2010-12-02 19:00...|1528.34000000

In [23]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `window` DESC
  """)\
  .show(20)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   13969.0|[2010-12-09 19:00...|            317.59|
|   14702.0|[2010-12-09 19:00...|             318.7|
|   17530.0|[2010-12-09 19:00...| 300.6399999999999|
|   13230.0|[2010-12-09 19:00...| 93.45000000000002|
|   13021.0|[2010-12-08 19:00...|            -20.28|
|   15358.0|[2010-12-08 19:00...| 624.5499999999998|
|   12797.0|[2010-12-08 19:00...|            254.03|
|   14355.0|[2010-12-08 19:00...|             174.4|
|   13117.0|[2010-12-08 19:00...|            186.03|
|   17671.0|[2010-12-08 19:00...|394.18000000000006|
|   15211.0|[2010-12-08 19:00...| 383.0700000000001|
|   17339.0|[2010-12-08 19:00...|              60.0|
|   17702.0|[2010-12-08 19:00...|-88.14999999999999|
|   15380.0|[2010-12-08 19:00...|230.10000000000002|
|   12872.0|[2010-12-08 19:00...|378.53000000000003|
|   16042.0|[2010-12-08 19:00...|            -

In [ ]:
# purchaseByCustomerPerHour.writeStream\
#     .format("console")\
#     .queryName("customer_purchases_2")\
#     .outputMode("complete")\
#     .start()

#check on Console